Alter file state table schema

In [0]:
def append_to_sys_path(folder: str, base_folder: str = "weather_bundle") -> str:
    """
    Append the specified folder to the system path if it is not already present.
    Allows importing packages from the folder.
    """
    import re
    import os
    import sys

    if not (match := re.search(rf"(.+[\\/]{base_folder})", os.getcwd())):
        raise FileNotFoundError(f"Could not find {base_folder} in current path: {os.getcwd()}")

    base_path = match.group(1)
    for root, dirs, _ in os.walk(base_path):
        if "src" in dirs:
            base_path = root
            break

    if (path := os.path.join(base_path, folder)) not in sys.path:
        sys.path.append(path)
    return path


append_to_sys_path("src/config")

In [0]:
dbutils.widgets.dropdown("dev_tests", "", ["", "True", "False"])  # noqa: F821

dev_tests = dbutils.widgets.get("dev_tests")  # noqa: F821

if not dev_tests:
    dev_tests = False

dev_tests = bool(dev_tests)

print(f"{dev_tests=}")
print(f"{type(dev_tests)=}")

In [0]:
from config import Config

if dev_tests:
    import os
    os.environ["ENV"] = "prd"
    os.environ["STAGE"] = "sandbox"

config = Config()
catalog = config["weather"]["catalog_name"]
bronze_schema = config["weather"]["bronze_schema_name"]
noaa: dict = config["weather"]["noaa"]
file_status_table_name = noaa["metrics"]["bronze_file_status_table_name"]

file_status_table = f"{catalog}.{bronze_schema}.{file_status_table_name}"

print(f"{file_status_table=}")

In [ ]:
# Add 'record_created_time' column to the table if it does not exist

column_name = "record_created_time"

# Check if the column exists
cols = [f.name for f in spark.table(file_status_table).schema.fields]
if column_name not in cols:
    spark.sql(f"ALTER TABLE {file_status_table} ADD COLUMNS ({column_name} TIMESTAMP)")
    print(f"Column '{column_name}' added to table '{file_status_table}'.")
else:
    print(f"Column '{column_name}' already exists in table '{file_status_table}'.")
